<a href="https://colab.research.google.com/github/XuTaoWang-2002/FlexEH-in-data-center/blob/main/clusterdata_analysis_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google trace analysis colab

This colab provides several example queries and graphs using [Altair](https://altair-viz.github.io/) for the 2019 Google cluster trace. Further examples will be added over time.

**Important:** in order to be able to run the queries you will need to:

1.   Use the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) to Create a Cloud Platform project if you do not already have one.
2.   [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
3.   [Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) APIs for the project.


In [1]:
#@title Please input your project id
import pandas as pd
import numpy as np
import altair as alt
from google.cloud import bigquery
# Provide credentials to the runtime
from google.colab import auth
from google.cloud.bigquery import magics

auth.authenticate_user()
print('Authenticated')
project_id = '4170242959' #@param {type: "string"}
# Set the default project id for %bigquery magic
magics.context.project = project_id

# Use the client to run queries constructed from a more complicated function.
client = bigquery.Client(project=project_id)

Authenticated


# Basic queries

This section shows the most basic way of querying the trace using the [bigquery magic](https://googleapis.dev/python/bigquery/latest/magics.html)

In [2]:
%%bigquery
SELECT capacity.cpus AS cpu_cap,
capacity.memory AS memory_cap,
COUNT(DISTINCT machine_id) AS num_machines
FROM `google.com:google-cluster-data`.clusterdata_2019_a.machine_events
GROUP BY 1,2

Query is running:   0%|          |

Downloading:   0%|          |

,cpu_cap,memory_cap,num_machines
0,NaN,NaN,459
1,1.000000,0.500000,1672
2,1.000000,1.000000,643
3,1.000000,0.250000,52
4,0.479492,0.250000,1
5,0.708984,0.333496,1417
6,0.708984,0.666992,480
7,0.708984,0.250000,1
8,0.958984,0.500000,179
9,0.958984,1.000000,73


In [4]:
datasets = list(client.list_datasets("google.com:google-cluster-data"))
for dataset in datasets:
    print("Dataset:", dataset.dataset_id)
    tables = list(client.list_tables(f"google.com:google-cluster-data.{dataset.dataset_id}"))
    for table in tables:
        print("  └──", table.table_id)

Dataset: clusterdata_2011_1
  └── job_events
  └── machine_attributes
  └── machine_events
  └── task_constraints
  └── task_events
  └── task_usage
Dataset: clusterdata_2019_a
  └── collection_events
  └── instance_events
  └── instance_usage
  └── machine_attributes
  └── machine_events
Dataset: clusterdata_2019_b
  └── collection_events
  └── instance_events
  └── instance_usage
  └── machine_attributes
  └── machine_events
Dataset: clusterdata_2019_c
  └── collection_events
  └── instance_events
  └── instance_usage
  └── machine_attributes
  └── machine_events
Dataset: clusterdata_2019_d
  └── collection_events
  └── instance_events
  └── instance_usage
  └── machine_attributes
  └── machine_events
Dataset: clusterdata_2019_e
  └── collection_events
  └── instance_events
  └── instance_usage
  └── machine_attributes
  └── machine_events
Dataset: clusterdata_2019_f
  └── collection_events
  └── instance_events
  └── instance_usage
  └── machine_attributes
  └── machine_events
Datas

In [3]:
table = client.get_table("google.com:google-cluster-data.clusterdata_2019_a.machine_events")
for field in table.schema:
    print(field.name, ":", field.field_type)
    if field.field_type == "RECORD":  # RECORD 表示结构体
        for subfield in field.fields:
            print("   └──", subfield.name, ":", subfield.field_type)

time : INTEGER
machine_id : INTEGER
type : INTEGER
switch_id : STRING
capacity : RECORD
   └── cpus : FLOAT
   └── memory : FLOAT
platform_id : STRING
missing_data_reason : INTEGER


In [5]:
table = client.get_table('google.com:google-cluster-data.clusterdata_2019_a.machine_attributes')
for field in table.schema:
    print(field.name, ":", field.field_type)
    if field.field_type == "RECORD":  # RECORD 表示结构体
        for subfield in field.fields:
            print("   └──", subfield.name, ":", subfield.field_type)

time : INTEGER
machine_id : INTEGER
name : STRING
value : STRING
deleted : BOOLEAN


In [6]:
table = client.get_table('google.com:google-cluster-data.clusterdata_2019_a.collection_events')
for field in table.schema:
    print(field.name, ":", field.field_type)
    if field.field_type == "RECORD":  # RECORD 表示结构体
        for subfield in field.fields:
            print("   └──", subfield.name, ":", subfield.field_type)

time : INTEGER
type : INTEGER
collection_id : INTEGER
scheduling_class : INTEGER
missing_type : INTEGER
collection_type : INTEGER
priority : INTEGER
alloc_collection_id : INTEGER
user : STRING
collection_name : STRING
collection_logical_name : STRING
parent_collection_id : INTEGER
start_after_collection_ids : INTEGER
max_per_machine : INTEGER
max_per_switch : INTEGER
vertical_scaling : INTEGER
scheduler : INTEGER


In [7]:
table = client.get_table('google.com:google-cluster-data.clusterdata_2019_a.instance_events')
for field in table.schema:
    print(field.name, ":", field.field_type)
    if field.field_type == "RECORD":  # RECORD 表示结构体
        for subfield in field.fields:
            print("   └──", subfield.name, ":", subfield.field_type)

time : INTEGER
type : INTEGER
collection_id : INTEGER
scheduling_class : INTEGER
missing_type : INTEGER
collection_type : INTEGER
priority : INTEGER
alloc_collection_id : INTEGER
instance_index : INTEGER
machine_id : INTEGER
alloc_instance_index : INTEGER
resource_request : RECORD
   └── cpus : FLOAT
   └── memory : FLOAT
constraint : RECORD
   └── name : STRING
   └── value : STRING
   └── relation : INTEGER


In [8]:
table = client.get_table('google.com:google-cluster-data.clusterdata_2019_a.instance_usage')
for field in table.schema:
    print(field.name, ":", field.field_type)
    if field.field_type == "RECORD":  # RECORD 表示结构体
        for subfield in field.fields:
            print("   └──", subfield.name, ":", subfield.field_type)

start_time : INTEGER
end_time : INTEGER
collection_id : INTEGER
instance_index : INTEGER
machine_id : INTEGER
alloc_collection_id : INTEGER
alloc_instance_index : INTEGER
collection_type : INTEGER
average_usage : RECORD
   └── cpus : FLOAT
   └── memory : FLOAT
maximum_usage : RECORD
   └── cpus : FLOAT
   └── memory : FLOAT
random_sample_usage : RECORD
   └── cpus : FLOAT
   └── memory : FLOAT
assigned_memory : FLOAT
page_cache_memory : FLOAT
cycles_per_instruction : FLOAT
memory_accesses_per_instruction : FLOAT
sample_rate : FLOAT
cpu_usage_distribution : FLOAT
tail_cpu_usage_distribution : FLOAT


In [9]:
%%bigquery
SELECT COUNT(DISTINCT collection_id) AS collections FROM
`google.com:google-cluster-data`.clusterdata_2019_a.collection_events;

Query is running:   0%|          |

Downloading:   0%|          |

,collections
0,5212740


In [11]:
%%bigquery
SELECT
  collection_id,
  COUNT(*) AS num_collect_id
FROM
  `google.com:google-cluster-data.clusterdata_2019_a.collection_events`
GROUP BY
  type
ORDER BY
  num_events DESC;

Query is running:   0%|          |

Downloading:   0%|          |

,type,num_events
0,2,5191645
1,3,5190786
2,0,4763891
3,7,3808267
4,6,1334160
5,1,463955
6,5,53548
7,9,695
8,8,455
9,10,24


In [18]:
%%bigquery
SELECT
   scheduling_class,
  COUNT(*) AS num_parent_id
FROM
  `google.com:google-cluster-data.clusterdata_2019_a.collection_events`
GROUP BY
   scheduling_class
ORDER BY
  num_parent_id DESC;

Query is running:   0%|          |

Downloading:   0%|          |

,scheduling_class,num_parent_id
0,1,13148910
1,0,4324638
2,2,1851514
3,3,1482379


# Cell level resource usage time series

This query takes a cell as input and plots a resource usage time-series for every hour of the trace broken down by tier.

In [ ]:
#@title Select a cell and a resource to plot the cell level usage series

def query_cell_capacity(cell):
  return '''
SELECT SUM(cpu_cap) AS cpu_capacity,
  SUM(memory_cap) AS memory_capacity
FROM (
  SELECT machine_id, MAX(capacity.cpus) AS cpu_cap,
    MAX(capacity.memory) AS memory_cap
  FROM `google.com:google-cluster-data`.clusterdata_2019_{cell}.machine_events
  GROUP BY 1
)
  '''.format(cell=cell)

def query_per_instance_usage_priority(cell):
  return '''
SELECT u.time AS time,
  u.collection_id AS collection_id,
  u.instance_index AS instance_index,
  e.priority AS priority,
  CASE
    WHEN e.priority BETWEEN 0 AND 99 THEN '1_free'
    WHEN e.priority BETWEEN 100 AND 115 THEN '2_beb'
    WHEN e.priority BETWEEN 116 AND 119 THEN '3_mid'
    ELSE '4_prod'
  END AS tier,
  u.cpu_usage AS cpu_usage,
  u.memory_usage AS memory_usage
FROM (
  SELECT start_time AS time,
    collection_id,
    instance_index,
    machine_id,
    average_usage.cpus AS cpu_usage,
    average_usage.memory AS memory_usage
  FROM `google.com:google-cluster-data`.clusterdata_2019_{cell}.instance_usage
  WHERE (alloc_collection_id IS NULL OR alloc_collection_id = 0)
    AND (end_time - start_time) >= (5 * 60 * 1e6)
) AS u JOIN (
  SELECT collection_id, instance_index, machine_id,
    MAX(priority) AS priority
  FROM `google.com:google-cluster-data`.clusterdata_2019_{cell}.instance_events
  WHERE (alloc_collection_id IS NULL OR alloc_collection_id = 0)
  GROUP BY 1, 2, 3
) AS e ON u.collection_id = e.collection_id
  AND u.instance_index = e.instance_index
  AND u.machine_id = e.machine_id
  '''.format(cell=cell)

def query_per_tier_utilization_time_series(cell, cpu_capacity, memory_capacity):
  return '''
SELECT CAST(FLOOR(time/(1e6 * 60 * 60)) AS INT64) AS hour_index,
  tier,
  SUM(cpu_usage) / (12 * {cpu_capacity}) AS avg_cpu_usage,
  SUM(memory_usage) / (12 * {memory_capacity}) AS avg_memory_usage
FROM ({table})
GROUP BY 1, 2 ORDER BY hour_index
  '''.format(table=query_per_instance_usage_priority(cell),
             cpu_capacity=cpu_capacity, memory_capacity=memory_capacity)

def run_query_utilization_per_time_time_series(cell):
  cell_cap = client.query(query_cell_capacity(cell)).to_dataframe()
  query = query_per_tier_utilization_time_series(
      cell,
      cell_cap['cpu_capacity'][0],
      cell_cap['memory_capacity'][0])
  time_series = client.query(query).to_dataframe()
  return time_series

cell = 'c' #@param ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
hourly_usage = run_query_utilization_per_time_time_series(cell)

# CPU graph
cpu = alt.Chart(hourly_usage).mark_area().encode(
        alt.X('hour_index:N'),
        alt.Y('avg_cpu_usage:Q'),
        color=alt.Color('tier', legend=alt.Legend(orient="left")),
        order=alt.Order('tier', sort='descending'),
        tooltip=['hour_index', 'tier', 'avg_cpu_usage']
  )
cpu.encoding.x.title = "Hour"
cpu.encoding.y.title = "Average CPU usage"
cpu.display()

# Memory graph
memory = alt.Chart(hourly_usage).mark_area().encode(
        alt.X('hour_index:N'),
        alt.Y('avg_memory_usage:Q'),
        color=alt.Color('tier', legend=alt.Legend(orient="left")),
        order=alt.Order('tier', sort='descending'),
        tooltip=['hour_index', 'tier', 'avg_memory_usage']
  )
memory.encoding.x.title = "Hour"
memory.encoding.y.title = "Average memory usage"
memory.display()

#Per machine resource usage distribution

This query takes a cell as input and plots a per-machine resource utilization CDF.

In [ ]:
#@title Select a cell and plot its per-machine resource utilization CDFs

# Functions to plot CDFs using Altair
def pick_quantiles_from_tall_dataframe(data, qcol, name=""):
  quantiles = pd.DataFrame([x for x in data[qcol]]).transpose()
  if name != "":
    quantiles.columns = data[name]
  return quantiles

# - data: a dataframe with one row and one or more columns of quantiles (results
#   returned from APPROX_QUANTILES)
# - qcols: a list of names of the quantiles
# - names: the names of each returned quantiles' columns.
def pick_quantiles_from_wide_dataframe(data, qcols, names=[]):
  quantiles = {}
  i = 0
  for qcol in qcols:
    col_name = qcol
    if i < len(names):
      col_name = names[i]
    quantiles[col_name] = data[qcol][0]
    i+=1
  return pd.DataFrame(quantiles)

# - quantiles: a dataframe where each column contains the quantiles of one
#   data set. The index (i.e. row names) of the dataframe is the quantile. The
#   column names are the names of the data set.
def plot_cdfs(quantiles, xlab="Value", ylab="CDF",
              legend_title="dataset", labels=[],
              interactive=False,
              title=''):
  dfs = []
  label = legend_title
  yval = range(quantiles.shape[0])
  esp = 1.0/(len(quantiles)-1)
  yval = [y * esp for y in yval]
  while label == xlab or label == ylab:
    label += '_'
  for col_idx, col in enumerate(quantiles.columns):
    col_label = col
    if col_idx < len(labels):
      col_label = labels[col_idx]
    dfs.append(pd.DataFrame({
        label: col_label,
        xlab: quantiles[col],
        ylab: yval
    }))
  cdfs = pd.concat(dfs)
  lines = alt.Chart(cdfs).mark_line().encode(
    # If you can draw a CDF, it has to be continuous real-valued
    x=xlab+":Q",
    y=ylab+":Q",
    color=label+":N"
  ).properties(
    title=title
  )
  if not interactive:
    return lines
  # Create a selection that chooses the nearest point & selects based on x-value
  nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=[ylab], empty='none')
  # Transparent selectors across the chart. This is what tells us
  # the y-value of the cursor
  selectors = alt.Chart(cdfs).mark_point().encode(
    y=ylab+":Q",
    opacity=alt.value(0),
  ).properties(
    selection=nearest
  )

  # Draw text labels near the points, and highlight based on selection
  text = lines.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest,
                       alt.Text(xlab+":Q", format=".2f"),
                       alt.value(' '))
  )

  # Draw a rule at the location of the selection
  rules = alt.Chart(cdfs).mark_rule(color='gray').encode(
    y=ylab+":Q",
  ).transform_filter(
    nearest.ref()
  )
  # Draw points on the line, and highlight based on selection
  points = lines.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
  )
  # Put the five layers into a chart and bind the data
  return alt.layer(lines, selectors, rules, text, points).interactive(
      bind_y=False)

# Functions to create the query

def query_machine_capacity(cell):
  return '''
SELECT machine_id, MAX(capacity.cpus) AS cpu_cap,
  MAX(capacity.memory) AS memory_cap
FROM `google.com:google-cluster-data`.clusterdata_2019_{cell}.machine_events
GROUP BY 1
  '''.format(cell=cell)

def query_top_level_instance_usage(cell):
  return '''
SELECT CAST(FLOOR(start_time/(1e6 * 300)) * (1000000 * 300) AS INT64) AS time,
  collection_id,
  instance_index,
  machine_id,
  average_usage.cpus AS cpu_usage,
  average_usage.memory AS memory_usage
FROM `google.com:google-cluster-data`.clusterdata_2019_{cell}.instance_usage
WHERE (alloc_collection_id IS NULL OR alloc_collection_id = 0)
  AND (end_time - start_time) >= (5 * 60 * 1e6)
  '''.format(cell=cell)

def query_machine_usage(cell):
  return '''
SELECT u.time AS time,
  u.machine_id AS machine_id,
  SUM(u.cpu_usage) AS cpu_usage,
  SUM(u.memory_usage) AS memory_usage,
  MAX(m.cpu_cap) AS cpu_capacity,
  MAX(m.memory_cap) AS memory_capacity
FROM ({instance_usage}) AS u JOIN
 ({machine_capacity}) AS m
ON u.machine_id = m.machine_id
GROUP BY 1, 2
  '''.format(instance_usage = query_top_level_instance_usage(cell),
             machine_capacity = query_machine_capacity(cell))

def query_machine_utilization_distribution(cell):
  return '''
SELECT APPROX_QUANTILES(IF(cpu_usage > cpu_capacity, 1.0, cpu_usage / cpu_capacity), 100) AS cpu_util_dist,
  APPROX_QUANTILES(IF(memory_usage > memory_capacity, 1.0, memory_usage / memory_capacity), 100) AS memory_util_dist
FROM ({table})
  '''.format(table = query_machine_usage(cell))

cell = 'd' #@param ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
query = query_machine_utilization_distribution(cell)
machine_util_dist = client.query(query).to_dataframe()
plot_cdfs(pick_quantiles_from_wide_dataframe(machine_util_dist, ['cpu_util_dist', 'memory_util_dist'], ['CPU', 'Memory']), xlab='x - resource utilization (%)', ylab="Probability (resource utilization < x)", interactive=True)